In [ ]:
!pip install transformers datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 96.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 54.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 29.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 108.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 89.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 28.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 78.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 29.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━

In [ ]:
import os
import time
import datetime
from google.colab import drive

import pandas as pd
import seaborn as sns
import numpy as np
import random



import torch
from torch.utils.data import Dataset, DataLoader, random_split, RandomSampler, SequentialSampler
torch.manual_seed(42)

from transformers import GPT2LMHeadModel,  GPT2Tokenizer, GPT2Config, GPT2LMHeadModel
from transformers import AdamW, get_linear_schedule_with_warmup

from datasets import load_dataset

In [ ]:

dataset = load_dataset('brunokreiner/genius-lyrics',split = 'train')
dataset

In [ ]:
dataset = dataset.select(range(0,1000))

In [ ]:
dataset.__len__()

In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2', bos_token='<|startoftext|>', eos_token='<|endoftext|>', pad_token='<|pad|>')

In [ ]:
print("The max model length is {} for this model, although the actual embedding size for GPT small is 768".format(tokenizer.model_max_length))
print("The beginning of sequence token {} token has the id {}".format(tokenizer.convert_ids_to_tokens(tokenizer.bos_token_id), tokenizer.bos_token_id))
print("The end of sequence token {} has the id {}".format(tokenizer.convert_ids_to_tokens(tokenizer.eos_token_id), tokenizer.eos_token_id))
print("The padding token {} has the id {}".format(tokenizer.convert_ids_to_tokens(tokenizer.pad_token_id), tokenizer.pad_token_id))

In [ ]:
class GPT2Dataset(Dataset):

  def __init__(self, txt_list, tokenizer, gpt2_type="gpt2", max_length=768):

    self.tokenizer = tokenizer
    self.input_ids = []
    self.attn_masks = []

    for txt in txt_list:

      encodings_dict = tokenizer('<|startoftext|>'+ txt["lyrics"] + '<|endoftext|>', truncation=True, max_length=max_length, padding="max_length")

      self.input_ids.append(torch.tensor(encodings_dict['input_ids']))
      self.attn_masks.append(torch.tensor(encodings_dict['attention_mask']))

  def __len__(self):
    return len(self.input_ids)

  def __getitem__(self, idx):
    return self.input_ids[idx], self.attn_masks[idx]

In [ ]:
dataset = GPT2Dataset(dataset, tokenizer, max_length=768)

# Split into training and validation sets
train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size

train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

print('{:>5,} training samples'.format(train_size))
print('{:>5,} validation samples'.format(val_size))

In [ ]:
train_dataloader = DataLoader(
    train_dataset,
    sampler = RandomSampler(train_dataset),
    batch_size = batch_size
)

validation_dataloader = DataLoader(
    val_dataset,
    sampler = SequentialSampler(val_dataset),
    batch_size = batch_size
)

In [ ]:
configuration = GPT2Config.from_pretrained('gpt2', output_hidden_states=False)
model = GPT2LMHeadModel.from_pretrained("gpt2", config=configuration)
model.resize_token_embeddings(len(tokenizer))

device = torch.device("cuda")
model.cuda()

seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [ ]:
epochs = 5
learning_rate = 5e-4
warmup_steps = 1e2
epsilon = 1e-8

sample_every = 100

In [ ]:
optimizer = AdamW(model.parameters(),
                  lr = learning_rate,
                  eps = epsilon
                )

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
total_steps = len(train_dataloader) * epochs

In [ ]:
def format_time(elapsed):
    return str(datetime.timedelta(seconds=int(round((elapsed)))))

In [ ]:
scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps = warmup_steps,
                                            num_training_steps = total_steps)

In [ ]:
total_t0 = time.time()

training_stats = []

model = model.to(device)

for epoch_i in range(0, epochs):

    # ========================================
    #               Training
    # ========================================

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    t0 = time.time()

    total_train_loss = 0

    model.train()

    for step, batch in enumerate(train_dataloader):

        b_input_ids = batch[0].to(device)
        b_labels = batch[0].to(device)
        b_masks = batch[1].to(device)

        model.zero_grad()

        outputs = model(  b_input_ids,
                          labels=b_labels,
                          attention_mask = b_masks,
                          token_type_ids=None
                        )

        loss = outputs[0]

        batch_loss = loss.item()
        total_train_loss += batch_loss

        # Get sample every x batches.
        if step % sample_every == 0 and not step == 0:

            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}. Loss: {:>5,}.   Elapsed: {:}.'.format(step, len(train_dataloader), batch_loss, elapsed))

            model.eval()

            sample_outputs = model.generate(
                                    bos_token_id=random.randint(1,30000),
                                    do_sample=True,
                                    top_k=50,
                                    max_length = 200,
                                    top_p=0.95,
                                    num_return_sequences=1
                                )
            for i, sample_output in enumerate(sample_outputs):
                  print("{}: {}".format(i, tokenizer.decode(sample_output, skip_special_tokens=True)))

            model.train()

        loss.backward()

        optimizer.step()

        scheduler.step()

    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)

    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epoch took: {:}".format(training_time))

    # ========================================
    #               Validation
    # ========================================

    print("")
    print("Running Validation...")

    t0 = time.time()

    model.eval()

    total_eval_loss = 0
    nb_eval_steps = 0

    # Evaluate data for one epoch
    for batch in validation_dataloader:

        b_input_ids = batch[0].to(device)
        b_labels = batch[0].to(device)
        b_masks = batch[1].to(device)

        with torch.no_grad():

            outputs  = model(b_input_ids,
#                            token_type_ids=None,
                             attention_mask = b_masks,
                            labels=b_labels)

            loss = outputs[0]

        batch_loss = loss.item()
        total_eval_loss += batch_loss

    avg_val_loss = total_eval_loss / len(validation_dataloader)

    validation_time = format_time(time.time() - t0)

    print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))

    # Record all statistics from this epoch.
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )

print("")
print("Training complete!")
print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))


======== Epoch 1 / 5 ========
Training...


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


  Batch   100  of    450. Loss: 1.74146568775177.   Elapsed: 0:00:48.
0:  bipartisan ( t he first party as a first party is the first party that i t o it a first party that i t o it a first party that i t o it a first party that i t o t o it that i t o it that i t o t o it that a first party that i t o it that an first party that i t o that a second party that t o that a second party that a second party that t o it that a third party that t o that a first party that t o that a second party that t o that a third party that a second party that t o that a first party that t o that a second party that a second party that t o it that a third party that t o that t that a second party that t o that a third party that t o that a second party that t o that a third party that t o that a second party that t o that a second party that t o


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


  Batch   200  of    450. Loss: 1.76533842086792.   Elapsed: 0:01:42.
0:  increasing it may it doesn t be they don t be one the most powerful tonnage tonnage tonnage tonnage tonnage tonnage tonnage tonnage tonnage tonnage tonnage tonnage tonnage tonnage tonnage tonnage tonnage tonnage tonnage tonnage tonnage tonnage tonnage tonnage tonnage tonnage tonnage tonnage tonnage tonnage tonnage tonnage tonnage tonnage tonnage tonnage tonnage tonnage tonnage tonnage tonnage tonnage tonnage tonnage tonnage tonnage tonnage tonnage tonnage tonnage tonnage tonnage tonnage tonnage tonnage tonnage tonnage tonnage tonnage tonnage


...

  Batch   200  of    450. Loss: 0.5618057250976562.   Elapsed: 0:01:45.
0: ismstrictly in the middle of the night he sits at home on the sofa having some coffee on the way he d rather not say it but there you go your dear stranger looks me in the eye and it s no longer too late and your lips are moving so warm i m falling in love but you re leaving me your lips moving so sweet i m falling in love but you re leaving me i m falling in love but you re leaving me i m falling in love but you re leaving me i m falling in love but you re leaving me i m falling in love but you re leaving me i m falling in love but you re leaving me i m falling in love but you re leaving me i m falling in love but you re leaving me i m falling in love but you re leaving me i m falling in love but you re leaving me i m falling in love but you re leaving me  i m falling in love but you re leaving me i m falling in love but you re leaving


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


  Batch   300  of    450. Loss: 0.8537077903747559.   Elapsed: 0:02:38.
0: ounf yolo yap c hanker man i ll be in my daddy s place forever you got a lot of trouble getting by i ll give you some more love tonight we can be having a laugh at the same time cause i ll tell you to stop playing em and do the things you do em so why don t you come over here if you wanna keep in touch with me i ll hold you close cause i can see you at your own pace cause i won t let you go if you wanna change your mind it ll be me doing the things you do em so why don t you come over here if you wanna keep in touch with me i ll hold you close cause i can see you at your own pace cause i won t let you go if you wanna change your mind it ll be me doing the things you do em so why don t you come over here if you wanna keep in touch with me i ll hold you close cause i can see you at your own pace cause i


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


  Batch   400  of    450. Loss: 1.8516812324523926.   Elapsed: 0:03:32.
0:  electionshe told me about a trip she went with a friend and i took her to bed a little after a little later but she said the world had changed she was in her 30s and i wanted to stay girl and you re your own man so we settled down to have one last night at the bar so i m on the phone she was in her 40s and i wanted to stay man and you re your own man so we settled down to have one last night at the bar so i m on the phone hey little you ll always be the kind that likes a place you know it can t be too far away so why should we spend the night anyway and she left me and my friends in the bed with him just to let her pass so i put that down and took a little stroll around the street she had a pretty little tattoo on her neck so i walked into her room i couldn t change the subject so i said goodbye and left then i said it in a voice i couldn t hear

  Average training loss: 1.09
  Training epoch took: 0:03:59

Run

In [ ]:
text = "I thought i knew them "
inputs = tokenizer.encode(text, return_tensors='pt').to(device)

In [ ]:
generated = model.generate(inputs, max_length=50, do_sample=True)
decoded = tokenizer.decode(generated[0], skip_special_tokens=True)
decoded

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'I thought i knew them  i wanted to look into the horizon and it seems to float underneath i think that maybe i m looking into another world if i close my eyes i d hear the sound of war s ringing i think i ve become a martyr'

In [ ]:
from huggingface_hub import notebook_login

In [ ]:
notebook_login()

In [ ]:
model.push_to_hub("lyrics_generator_asg_gpt2")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/_commit_api.py:232: UserWarning: About to update multiple times the same file in the same commit: 'pytorch_model.bin'. This can cause undesired inconsistencies in your repo.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/_commit_api.py:232: UserWarning: About to update multiple times the same file in the same commit: 'generation_config.json'. This can cause undesired inconsistencies in your repo.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/_commit_api.py:232: UserWarning: About to update multiple times the same file in the same commit: 'config.json'. This can cause undesired inconsistencies in your repo.
  warnings.warn(


Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

pytorch_model.bin:   0%|          | 0.00/498M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/498M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/harsh098mumbai/lyrics_generator_asg_gpt2/commit/ecb456d51361cc1882ab0dcf3d4ae7b913c37e32', commit_message='Upload model', commit_description='', oid='ecb456d51361cc1882ab0dcf3d4ae7b913c37e32', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
tokenizer.push_to_hub("lyrics_generator_asg_gpt2")

CommitInfo(commit_url='https://huggingface.co/harsh098mumbai/lyrics_generator_asg_gpt2/commit/a06a5b1ec0645f9af8b1d3750edad01f1e62d341', commit_message='Upload tokenizer', commit_description='', oid='a06a5b1ec0645f9af8b1d3750edad01f1e62d341', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
torch.save(tokenizer,'tokenizer.pt')

In [ ]:
!pip install gradio

NotImplementedError: ignored

In [ ]:
import gradio as gr

ModuleNotFoundError: ignored

In [ ]:
from huggingface_hub import HfApi, snapshot_download
import json

In [ ]:
def push_to_hub(repo_id,
                model,
                tokenizer,
                local_directory
                ):


  _, repo_name = repo_id.split("/")
  api = HfApi()

  # Step 1: Create the repo
  repo_url = api.create_repo(
        repo_id=repo_id,
        exist_ok=True,
  )


  torch.save(model, "models/model.pt")

  torch.save(tokenizer,"models/tokenizer.pt")

    # Step 7. Push everything to the Hub
  api.upload_folder(
        repo_id=repo_id,
        folder_path=local_directory,
        path_in_repo=".",
  )

  print(f"Your model is pushed to the Hub. You can view your model here: {repo_url}")

In [ ]:
push_to_hub("harsh098mumbai/lyrics_generator_asg_gpt2",model,tokenizer,'models')

model.pt:   0%|          | 0.00/510M [00:00<?, ?B/s]

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

Your model is pushed to the Hub. You can view your model here: https://huggingface.co/harsh098mumbai/lyrics_generator_asg_gpt2
